# Intro to ML Project 3
## Decision Trees Classification

In [1]:
# Importing Libraries
%matplotlib inline
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import copy

In [2]:
def str_to_int(df, col_indx):
    """
    Function to convert string value to unique integers in place and return dictionary 
    Input: df :pd.DataFrame
           col_index: column index
    output: dictionary: dict to map str : int
    """
    col = df[col_indx]
    row = len(col)
    keys = pd.unique(df[col_indx])
    dictionary = dict(zip(keys, np.linspace(0,len(keys)-1,len(keys),dtype=int)))
    for j in range(row):
        df[col_indx][j] = dictionary[df[col_indx][j]]
    return dictionary

In [3]:
# Breast Cancer (classification)
# the last column is the target
file_loc = './Data_Set/breast-cancer-wisconsin.data'
bc = pd.read_csv(file_loc, header = None)
bc = bc.iloc[:, 1:] # dropping 1st column since they are unique identifiers.
bc.columns = [i for i in range(len(bc.columns))] #fixing the columns index num
#imputing missing values
NAs = []
for i in range(10):
    for j in range(698):
        if bc[i][j]== '?':
            NAs.append([i,j])
bc_vals = bc[5][bc[5]!='?'].astype('int')
samples = np.random.choice(bc_vals, 16).copy()
for k, i in enumerate(NAs):
    bc.iloc[i[1],i[0]] = samples[k]
for col in range(bc.shape[1]):        # converting all num to np.int64
    bc[col] = bc[col].astype('int64')
bc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       699 non-null    int64
 1   1       699 non-null    int64
 2   2       699 non-null    int64
 3   3       699 non-null    int64
 4   4       699 non-null    int64
 5   5       699 non-null    int64
 6   6       699 non-null    int64
 7   7       699 non-null    int64
 8   8       699 non-null    int64
 9   9       699 non-null    int64
dtypes: int64(10)
memory usage: 54.7 KB


In [4]:
# car (classification)
# the last column is the target (6)
file_loc = './Data_Set/car.data'
car = pd.read_csv(file_loc, header = None)
[str_to_int(car, i) for i in range(7)] # converting string to integer
for col in range(car.shape[1]):        # converting all num to np.int64
    car[col] = car[col].astype('int64')
car

,0,1,2,3,4,5,6
0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,2,0
3,0,0,0,0,1,0,0
4,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...
1723,3,3,3,2,1,1,3
1724,3,3,3,2,1,2,2
1725,3,3,3,2,2,0,0
1726,3,3,3,2,2,1,3


In [5]:
#segmentation (classification)
file_loc = './Data_Set/segmentation.data'
segmentation = pd.read_csv(file_loc, skiprows=[0,1,2,3,4], header=None)
str_to_int(segmentation, 0)

# col 3 is all identical and has no info. will frop this
segmentation.drop([3,4,5], axis= 1, inplace=True)
target = segmentation[0]
segmentation = pd.concat([segmentation.iloc[:,1:],target], axis = 1)
segmentation.columns = [i for i in range(segmentation.shape[1])]
segmentation.iloc[:,16] = segmentation.iloc[:,16].astype(int)
segmentation

<ipython-input-2-346e83ab41dd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_indx][j] = dictionary[df[col_indx][j]]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,140.0,125.0,0.277778,0.062963,0.666667,0.311111,6.185185,7.333334,7.666666,3.555556,3.444444,4.444445,-7.888889,7.777778,0.545635,-1.121818,0
1,188.0,133.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817,0
2,105.0,139.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946,0
3,34.0,137.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272,0
4,39.0,111.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,36.0,243.0,1.888889,1.851851,2.000000,0.711110,13.333333,9.888889,12.111111,18.000000,-10.333333,-3.666667,14.000000,18.000000,0.452229,2.368310,6
206,186.0,218.0,1.166667,0.744444,1.166667,0.655555,13.703704,10.666667,12.666667,17.777779,-9.111111,-3.111111,12.222222,17.777779,0.401347,2.382683,6
207,197.0,236.0,2.444444,6.829628,3.333333,7.599998,16.074074,13.111111,16.666668,18.444445,-8.888889,1.777778,7.111111,18.555555,0.292729,2.789800,6
208,208.0,240.0,1.055556,0.862963,2.444444,5.007407,14.148149,10.888889,13.000000,18.555555,-9.777778,-3.444444,13.222222,18.555555,0.421621,2.392487,6


# Cross Validation Tests

In [6]:
# cross_validation (stratefied) 5 fold:
def stratefied_cross_validation(df, target, k_fold = 5):
    """
    Function to split data for stratefied k-fold cv testing 
    where each group have same class representation, plus 10% validation set
    Input: original data, target feature id
    Output:'indeces' of stratefied k-fold cv sets, and tuning set 
    """
    N = len(df)
    index = [index for index in df.index]
    np.random.shuffle(index)
    validation_set = index[:int(N/10)]
    remain= index[int(N/10):]
    # among the remain, group the data into each classifications
    k_splits = [[] for _ in range(k_fold)]
    classes = np.unique(df[target])
    for c in classes:
        group = [idx for idx in remain if df.iloc[idx,target]==c]
        np.random.shuffle(group)
        group_k_splits = np.array_split(group, k_fold)
        np.random.shuffle(group_k_splits)
        for k in range(k_fold):
            holdout = group_k_splits[k]
            np.random.shuffle(holdout)
            k_splits[k].extend(holdout)
            
    return k_splits, validation_set


In [7]:
# Wrapper to generate dataset from index list
def k_fold_cv_sets(k_splits):
    """ 
    Based on the k_splits indices, returns 1 hold out set for testing
    and the rest for training
    Output: k sets of (train_set, test_set)
    """
    k_fold = len(k_splits)
    k_fold_idx = []
    for k in range(k_fold):
        test_set = k_splits[k]
        training_idx = [i for i in range(k_fold) if i !=k]
        train_set = []
        for i in training_idx:
            train_set.extend(k_splits[i])
        k_fold_idx.append((train_set, test_set))
    return k_fold_idx

In [8]:
def accuracy(test_y, y_hats):
    """
    Input: data_y, true value vector, y_hat, prediction vector
    Output: accuracy as number of correct prediction/ total population
    """
    correct = 0
    N = len(test_y)
    for i in range(len(test_y)):
        correct += test_y[i] == y_hats[i]
    return correct/N

# Helper Functions

In [9]:
def entrophy(df, feature):
    """
    Input: df: pd.DataFrame
           feature: feature id
    Output: entropy of given DataFrame based on the seleced feature
    """
    if len(df)<=1:
        return 0
    classes = pd.unique(df[feature])
    p = df[feature].value_counts().values/len(df[feature])
    return -sum(p*np.log2(p))

In [10]:
def continuous(df, feature):
    """
    Input: df: pd.DataFrame
           feature: feature id ( column num)
    Output: returns True if the feature is continuous, False if descrete
    """
    if df[feature].any:
        if (df[feature].dtype == 'int64')or(df[feature].dtype == 'int32'):
            return False
        else:
            return True
    else:
        return False

In [11]:
def most_common_label(df, target, regression = False):
    """
    Input: df: pd.DataFrame
           target: target id
    Output: returns the most commonly present target label for classification
            return the mean of target values for continuous values.
    """
    if regression:
        # if target is continuous take the mean
        label = np.mean(df[target])
    else:
        # if target is descrete valuestake the plural voting
        label = df[target].value_counts(ascending=False).keys()[0]
    return label

# Metric Calculations Functions

In [12]:
def split_entrophy(df, feature, target,tol=1e-4, binary_split=False):
    """
    Input: df, pd.DataFrame
           feature: feature id
           feature_val: feature value
           binary_split: default None, used as threshold to binary split
    Output: entropy after DataFrame was split on the given feature value
            information value for the feature being concidered
    """
    MAX = float('inf')
    
    def partial_entrophy(df,subset, target):
         return (len(subset)/len(df))*entrophy(subset, target)
    
    def partial_information_value(df, subset):
        # if this partition create no subset, no information value left
        if (len(subset)==0) | (len(df)==0):
            return 0  
        else:
            p = len(subset)/len(df)
            return -p*np.log2(p)
    
    binary_split = continuous(df, feature)
    best_split_point = None
    information_value = 0
    
    if binary_split == False:  # for descrete value

        feature_vals = np.sort(df[feature].unique())
        post_split_entrophy = 0
        information_value = 0
        for feature_val in feature_vals:
            subset = df[df[feature]==feature_val]
            post_split_entrophy += partial_entrophy(df,subset, target)
            information_value += partial_information_value(df, subset)
            
    else:  # for continuous value
        MIN_entrophy = MAX       
        df_sorted = df.sort_values(by=feature)
        index = df_sorted.index
        for i in range(len(index)-1):
            split_point = df_sorted[feature][index[i]]
            
            if df_sorted[target][index[i]]!= df_sorted[target][index[i+1]]:
                
                entrophy_at_split_i = 0
                information_value_i = 0
                
                # get partial entrophy for left of the split
                subset_L = df_sorted[df_sorted[feature]<split_point]
                information_value += partial_information_value(df, subset_L)
                entrophy_at_split_i+= partial_entrophy(df,subset_L, target)
                
                # get partial entrophy for right of the split
                subset_R = df_sorted[df_sorted[feature]>=split_point]
                information_value_i += partial_information_value(df, subset_R)
                entrophy_at_split_i+= partial_entrophy(df,subset_R, target)
                
                if entrophy_at_split_i < MIN_entrophy:
                    MIN_entrophy = entrophy_at_split_i  
                    best_split_point = split_point
                    information_value = information_value_i
                    
        post_split_entrophy = MIN_entrophy 
        
    return post_split_entrophy, information_value, best_split_point
        


In [13]:
def splitFeature(df, target):
    """
    Input: df: pd.DataFrame
           target: target id
    Output: returns col index of the highest post_split_entrophy
    """
    features = [col for col  in df.columns if col!=target]
    best_feature = None
    best_gain_ratio = float('inf')
    best_split = None
    pre_split_entrophy = entrophy(df, target) # entrophy before the split
    
    if len(features) == 0:
        return best_feature, best_gain_ratio, best_split

    for feature in features:
        post_split_entrophy, information_value, best_split_point = split_entrophy(df, feature, target, continuous(df, feature))
        gain = pre_split_entrophy - post_split_entrophy
        gain_ratio = gain/information_value
        if gain_ratio < best_gain_ratio:
            best_feature = feature
            best_gain_ratio = gain_ratio
            best_split = best_split_point
    
    return best_feature, best_gain_ratio, best_split

# Node and Tree

In [14]:
# Node class for tree building
class Node:
    def __init__(self):
        self.children = {}
        self.label = None
        self.feature = None 
        self.feature_values = []
        self.bin = []
        self.split_point = None
        self.parent = self
        #self.parent_feature = None
        #self.parent_feature_value = None
        self.isLeaf = False
        self.pruned = False  # for Pruning 
        self.gain_ratio = None

In [15]:
# Tree building
def ID3(df, target, tol= 1e-4):
    node = Node()
    node.label = most_common_label(df,target)
    node.bin = df.iloc[:,target]
    features = [col for col  in df.columns if col!=target]

    if (len(df.iloc[:,target].unique())<=1):  # if the node is pure, return node
        node.label = most_common_label(df,target)
        node.isLeaf = True
        return node

    best_feature, best_gain_ratio, best_split = splitFeature(df, target)
    
    node.feature = best_feature
    node.gain_ratio = best_gain_ratio
    node.split_point = best_split
    
    if best_feature == None: #there is no feature to split on. The node is already pure.
        #node.feature = node.parent.feature # the best feature be the one that was used by the parent
        node.label = most_common_label(df,target)
        node.isLeaf = True
        return node

    if continuous(df, best_feature): 
        # for binary split, L= 0, R= 1
        node.feature_values = [0, 1]
        subset_L = df[df[best_feature]< best_split]
        subset_R = df[df[best_feature]>= best_split]
        
        if subset_L.empty or subset_R.empty:
            # if nothing is less than the split point, take the median of the whole set
            node.label = most_common_label(df,target)
            node.isLeaf = True
            return node
        
        elif len(subset_L)<=1:
            node.label = most_common_label(subset_L,target)
            node.isLeaf = True
            return node
        
        elif len(subset_R)<=1:
            node.label = most_common_label(subset_R,target)
            node.isLeaf = True
            return node
        
        else:
            child_L = ID3(subset_L, target)
            child_L.bin = subset_L.iloc[:,target]
            child_L.parent = node
            node.children[0] = child_L
            child_R = ID3(subset_R, target)
            child_R.bin = subset_R.iloc[:,target]
            child_R.parent = node
            node.children[1] = child_R
        return node

    else:
        node.feature_values = df[best_feature].unique()  # all class labels
        for feature_val in node.feature_values:
            subset = df[df[best_feature]==feature_val]
            child = ID3(subset, target)
            child.parent = node
            child.bin = subset.iloc[:,target]
            node.children[feature_val] = child    

        return node

## Prediction 

In [16]:
def predict(node, df, sample, target, continuous_val = False):
    """
    This function takes input and traverse through the DT starting at the root and returns the\
    leaf if reached, if not take the most common value at the last node reached. 
    For node 
    Input: trained DT root node
           target: target id
           sample to be predicted
    Output: prediction
    """
    if node.isLeaf:
        # if node is a leaf and not pruned,return its label
        return node.label
            
    # for continuous value
    if continuous(df, node.feature):           
        if (sample[node.feature] < node.split_point) and (node.children[0].pruned == False):
            return predict(node.children[0], df, sample, target)
        elif (sample[node.feature] >= node.split_point) and (node.children[1].pruned == False):
            return predict(node.children[1], df, sample, target)
        else:
            return node.label
        
    #for discrete feature values
    else:
        #if the children exsist and is not pruned
        if (sample[node.feature] in node.children) and (node.children[int(sample[node.feature])].pruned== False): 
                # go down further down the tree 
                return predict(node.children[sample[node.feature]], df, sample, target)
        else:
            # if the children node does not have the sample value, or is pruned return the current label
            return node.label
            

In [17]:
def ID3_cross_validation_test(df, target, prune = True, score = accuracy , k= 5):
    """
    This function conducts stratefied k-hold cv tes
    Input:
        data_X : matrix, model feature set
        target:target feature id
        score: evaluation function
    Out:
        average score over k-folds
    """
    k_splits, validation_set = stratefied_cross_validation(df, target, k_fold = k)
    k_fold_idx = k_fold_cv_sets(k_splits)
    unpruned = []
    pruned = []
    for i in range(k):
        train_idx, test_idx =  k_fold_idx[i]
        train_X= df.iloc[train_idx]
        test_y = df.iloc[test_idx][target].values
        y_hats = []
        node = ID3(train_X, target) # build DT
        # test for unpruned tree
        for test_id in test_idx:
            sample = df.iloc[test_id, :]
            y_hats.append(predict(node, df, sample, target))
        unpruned.append(score(list(test_y), list(y_hats)))
        if prune:
            prune_tree(node, validation_set, df, target, score)
            for test_id in test_idx:
                sample = df.iloc[test_id, :]
                y_hats.append(predict(node, df, sample, target))
                pruned.append(score(list(test_y), list(y_hats)))
            return np.mean(unpruned), np.mean(pruned)
        
    return np.mean(unpruned)

In [18]:
df = bc
target = 9
k_splits, validation_set = stratefied_cross_validation(df, target, k_fold = k)
k_fold_idx = k_fold_cv_sets(k_splits)

i = 0
train_idx, test_idx =  k_fold_idx[i]
train_X= df.iloc[train_idx]
test_y = df.iloc[test_idx][target].values
print(test_y)
df.iloc[test_idx, target].values

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4]


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int64)

In [19]:
tree = None
def prune_tree(node, validation_set, df, target, score = accuracy):
    """
    Conduct pruning based on test score using validation set. 
    If score improves with pruning a leaf node, it will be pruned, o.w. kept. 
    """
    #tree = copy.deepcopy(node)
    global tree
    tree = node
    def prune_test(node,validation_set, df, target, score = accuracy):
        """
        conduct sub_test for pre, post pruning
        """
        y_hats = []
        test_y = df[target][validation_set].values
        for val_id in validation_set:
            sample = df.iloc[val_id]
            y_hats.append(predict(node, df, sample, target))
        test_score = score(list(test_y), list(y_hats))
        return test_score
    
    def prune_node(node, validation_set, df, target, score):
        if node.isLeaf:
            #try pruning
            pre_pruning_test = prune_test(tree,validation_set, df, target, score = score)
            node.pruned = True # mark the node pruned
            node.parent.isLeaf = True # make its parent a leaf
            post_pruning_test = prune_test(tree,validation_set, df, target, score = score)
            if pre_pruning_test >= post_pruning_test:
                node.pruned = False # don't prune this node 
                node.parent.isLeaf = False
            return
        
        children = [child for child in node.children.values()]
        for child in children:
            prune_node(child, validation_set, df, target, score = accuracy)
        
        pre_pruning_test = prune_test(tree,validation_set, df, target, score = score)
        node.pruned = True # mark the node pruned
        node.parent.isLeaf = True # make its parent a leaf
        post_pruning_test = prune_test(tree,validation_set, df, target, score = score)
        if pre_pruning_test >= post_pruning_test:
            node.pruned = False
            node.parent.isLeaf = False
    
    prune_node(tree, validation_set, df, target, score)
    
        

In [20]:
df = car
target = 6
k_splits, validation_set = stratefied_cross_validation(df, target,k_fold= 5)
k_fold_idx = k_fold_cv_sets(k_splits)
i = 0
train_idx, test_idx =  k_fold_idx[i]
train_X= df.iloc[train_idx]
test_y = df.iloc[test_idx][target].values
y_hats = []
#node = ID3(df, target) # build DT

#ID3_cross_validation_test(df, target, prune=True, score = accuracy, k =1 )
 

In [21]:
def dfs(node):
    global pruned
    global unprunedd
    if node.isLeaf and node.pruned:
        pruned +=1
        print('pruned')
    if node.isLeaf and node.pruned==False:
        print('unpruned')
        unpruned +=1
    else:
        children = [child for child in node.children.values()]
        for child in children:
            dfs(child)
    print(pruned, unpruned)

In [24]:
df = bc
target = 9
ID3_cross_validation_test(df, target, prune= True, score = accuracy )

<ipython-input-13-9bb55841cdc5>:19: RuntimeWarning: invalid value encountered in double_scalars
  gain_ratio = gain/information_value


(0.873015873015873, 0.8730158730158734)

In [23]:
df = car
target = 6
ID3_cross_validation_test(df, target, prune=False, score = accuracy )

<ipython-input-13-9bb55841cdc5>:19: RuntimeWarning: invalid value encountered in double_scalars
  gain_ratio = gain/information_value


0.6838135872701788

In [29]:
df = segmentation
target= 16
#node = ID3(df, target)
ID3_cross_validation_test(df, target, prune=False, score = accuracy )

<ipython-input-14-7a9e88d90747>:19: RuntimeWarning: invalid value encountered in double_scalars
  gain_ratio = gain/information_value


0.20872769241190295

In [38]:
df = car
target = 6
k_splits, validation_set = stratefied_cross_validation(df, target, k_fold = k)
k_fold_idx = k_fold_cv_sets(k_splits)
i = 0
train_idx, test_idx =  k_fold_idx[i]
train_X= df.iloc[train_idx]
test_y = df.iloc[test_idx,target].values
train_y = df.iloc[train_idx, target].values
node_car = ID3(train_X, target, tol=1) # build DT
# test for unpruned tree
for test_id in test_idx:
    sample = df.iloc[test_id, :]
    y_hats.append(predict(node_car, df, sample, target))

<ipython-input-13-9bb55841cdc5>:19: RuntimeWarning: invalid value encountered in double_scalars
  gain_ratio = gain/information_value


NameError: name 'no' is not defined